### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Gastroesophageal_reflux_disease_(GERD)/GSE43580'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
# Based on the series title and overall design, it mentions gene expression profiles
is_gene_available = True

# Check for variable availability and data type conversion
# Gastroesophageal_reflux_disease_(GERD)
for key, values in clinical_data.items():
    if any(isinstance(value, str) and "gastroesophageal reflux disease" in value.lower() for value in values):
        trait_row = key
        break

# Age
for key, values in clinical_data.items():
    if any(isinstance(value, str) and "age at excision (years)" in value.lower() for value in values):
        age_row = key
        break

# Gender
for key, values in clinical_data.items():
    if any(isinstance(value, str) and "gender" in value.lower() for value in values):
        gender_row = key
        break

# Data Type Conversion Functions
def convert_trait(value):
    try:
        value = str(value).split(':')[-1].strip().lower()
        if "gastroesophageal reflux disease" in value:
            return 1
        return 0
    except:
        return None

def convert_age(value):
    try:
        value = str(value).split(':')[-1].strip()
        return float(value) if value != "NA" else None
    except:
        return None

def convert_gender(value):
    try:
        value = str(value).split(':')[-1].strip().lower()
        if value == "female":
            return 0
        if value == "male":
            return 1
        return None
    except:
        return None

# Save Metadata
save_cohort_info('GSE43580', './preprocessed/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if all([trait_row, age_row, gender_row]):
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Gastroesophageal_reflux_disease_(GERD)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Gastroesophageal_reflux_disease_(GERD)/trait_data/GSE43580.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
else:
    missing_features = [ft for ft, val in zip(['trait', 'age', 'gender'], [trait_row, age_row, gender_row]) if val is None]
    print(f"Required clinical features are not available: {', '.join(missing_features)}.")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine identifier_key and gene_symbol_key from gene_annotation preview
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
# Based on the series title and overall design, it mentions gene expression profiles
is_gene_available = True

# Check for variable availability and data type conversion
# Gastroesophageal_reflux_disease_(GERD)
for key, values in sample_characteristics_dict.items():
    if any(isinstance(value, str) and "gastroesophageal reflux disease" in value.lower() for value in values):
        trait_row = key
        break

# Age
for key, values in sample_characteristics_dict.items():
    if any(isinstance(value, str) and "age at excision (years)" in value.lower() for value in values):
        age_row = key
        break

# Gender
for key, values in sample_characteristics_dict.items():
    if any(isinstance(value, str) and "gender" in value.lower() for value in values):
        gender_row = key
        break

# Data Type Conversion Functions
def convert_trait(value):
    try:
        value = str(value).split(':')[-1].strip().lower()
        if "gastroesophageal reflux disease" in value:
            return 1
        return 0
    except:
        return None

def convert_age(value):
    try:
        value = str(value).split(':')[-1].strip()
        return float(value) if value != "NA" else None
    except:
        return None

def convert_gender(value):
    try:
        value = str(value).split(':')[-1].strip().lower()
        if value == "female":
            return 0
        if value == "male":
            return 1
        return None
    except:
        return None

# Save Metadata
save_cohort_info('GSE43580', './preprocessed/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None and age_row is not None and gender_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Gastroesophageal_reflux_disease_(GERD)', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Gastroesophageal_reflux_disease_(GERD)/trait_data/GSE43580.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
else:
    missing_features = [ft for ft, val in zip(['trait', 'age', 'gender'], [trait_row, age_row, gender_row]) if val is None]
    print(f"Required clinical features are not available: {', '.join(missing_features)}.")


### Step 7: Data Normalization and Merging

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Gastroesophageal_reflux_disease_(GERD)/gene_data/GSE43580.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Gastroesophageal_reflux_disease_(GERD)')

# 4. Save the cohort information.
save_cohort_info('GSE43580', './preprocessed/Gastroesophageal_reflux_disease_(GERD)/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Gastroesophageal_reflux_disease_(GERD)/GSE43580.csv'
    unbiased_merged_data.to_csv(csv_path)
